In [1]:
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import os
from glob import glob
import pandas as pd

In [2]:
data_path = 'D:\\Data\\text_summarization\\hvg\\'

In [3]:
files = glob(data_path + '*.txt')
files[-5:]

['D:\\Data\\text_summarization\\hvg\\_tudomany_20150411_MTAmunkatars_belathato_idon_belul_lehet.txt',
 'D:\\Data\\text_summarization\\hvg\\_tudomany_20151219_Az_elso_magyarorszagi_tudoaultetes.txt',
 'D:\\Data\\text_summarization\\hvg\\_tudomany_20151227_Elhunyt_a_Gproteinek_Nobeldijas_felfede.txt',
 'D:\\Data\\text_summarization\\hvg\\_tudomany_20160310_Brit_tudosok_az_EU_mellett.txt',
 'D:\\Data\\text_summarization\\hvg\\_tudomany_y.txt']

In [4]:
# from dateutil.parser import parse

def read_file(file):
    res = {}
    with open(file, 'r', encoding='utf8') as f:
        # title, description, tags, published_time, modified_time, section, content
        data = [line.strip() for line in f.readlines()]
    res['title'] = data[0]
    res['description'] = data[1]
    res['tags'] = data[2]
    res['published_time'] = data[3]
    res['modified_time'] = data[4]
    res['section'] = data[5]
    res['content'] = '\n'.join(data[6:])
    return res

In [5]:
def get_content(raw_content):
    content = ''
    html = BeautifulSoup(raw_content, 'html.parser')
    if len([tag.name for tag in html.find_all()]) == 1:
        content = html.text.strip()
    else:
        for p in html.find_all('p'):
            content = content + '\n' + p.text.strip()
    return content.strip()

In [6]:
column_names = ['title', 'description', 'tags', 'published_time', 'modified_time', 'section', 'content']
df = pd.DataFrame(columns = column_names)

In [15]:
for i, file in tqdm(enumerate(files[:])):
    data = read_file(file)
    raw_content = data['content']
    content = get_content(raw_content)
    df.loc[i] = [data['title'],
                 data['description'],
                 data['tags'],
                 data['published_time'],
                 data['modified_time'],
                 data['section'],
                 content]

In [13]:
read_file(files[-2000])['content']

'<div class="article-content entry-content">\n<p>Az ellenzékben lévő Északi Liga pártja tüntetést hirdetett meg vasárnapra az olaszországi Vignola városában, miután ott élő muzulmánok olasz fiatalokat fenyegettek meg fegyverrel.\xa0Az Északi Liga az érintett muzulmánok azonnali kiutasítását követeli Olaszországból.</p>\n<p>Az Emilia-Romagna tartomány huszonötezer lakosú városában tartózkodási engedéllyel élő huszonéves muzulmán férfiak öt olasz fiatalkorút fenyegettek meg az utóbbi napokban. Négy marokkói bevándorlóról van szó.\xa0 Öt 16-17 éves olasz fiút riasztópisztollyal ijesztgettek az Iszlám Állam (IÁ) terroristáinak videóin látott kivégzéseket utánozva. Azt a kérdést is a fiataloknak szegezték, hogy Istenben vagy Allahban hisznek-e. A fiatalok szülei jelentették az esetet a hatóságoknak. A muzulmán férfiak ezzel egy időben önként jelentkeztek a csendőrségen azt állítva, hogy tréfáról volt szó. A pisztolyt is átadták. Az ügyben vizsgálat indult.</p>\n<p>"Bizonyos tettek elfogadha

In [16]:
df.head()

,title,description,tags,published_time,modified_time,section,content
0,Kamion mentése miatt terelés az M60-ason,Lezárták az M60-as autópályát egy kamion műsza...,"kamion,M60-as autópálya",2011-12-14T13:32:00.0000000+01:00,2011-12-14T13:34:33.0000000+01:00,Baleset rovat,Lezárták az M60-as autópályát Bóly és Mohács k...
1,Szivárgó nitrogén miatt zárták le az M60-ast,Tartályautóból szivárgó nitrogén miatt kellett...,"nitrogén,M60-as autópálya",2011-12-14T17:15:00.0000000+01:00,2011-12-14T17:25:02.0000000+01:00,Baleset rovat,Az Országos Katasztrófavédelmi Főigazgatóságna...
2,Szén borítja a 34-es utat,"Lezárták a 34-es főút tiszafüredi szakaszát, m...","baleset,közlekedés,szén,Tiszafüred,34-es út",2011-12-14T20:36:00.0000000+01:00,2011-12-14T20:26:51.0000000+01:00,Baleset rovat,Szerdán este ütközött össze egy szenet szállít...
3,Árokba dőlt egy busz a 85-ös úton,Árokba dőlt egy diákokat szállító busz a 85-ös...,"baleset,85-ös út,Kapuvár",2011-12-15T16:15:00.0000000+01:00,2011-12-15T16:52:15.0000000+01:00,Baleset rovat,"A forgalmat lassították, a helyszínre kivonult..."
4,Gázolás miatt késnek a vonatok a Budapest-Szol...,Gázolás miatt fél órás késéssel közlekednek a ...,"vonat,gázolás,késés",2011-12-15T15:45:00.0000000+01:00,2011-12-15T15:40:01.0000000+01:00,Baleset rovat,A közlemény szerint Szemeretelep megállóhelyné...


In [17]:
df.section.unique()

array(['Baleset rovat', 'Világ rovat', 'Hírek rovat', 'w.hvg.hu rovat',
       'Tudomány rovat'], dtype=object)